In [1]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
import numpy as np
from pyspark.mllib.linalg.distributed import *
from pyspark.mllib.linalg import *
from scipy.sparse import *
sc = SparkContext.getOrCreate()
spark = SparkSession(sc)

In [2]:
icm = sc.textFile("../icm.csv")
icm = icm.map(lambda l: l.split(','))\
            .filter(lambda line: line[0] != 'itemId')\
            .map(lambda x: (int(x[0]), int(x[1]), 1))
trainSet = sc.textFile("../train.csv")
trainSet = trainSet.map(lambda l: l.split(','))
trainSet = trainSet.filter(lambda line: 'userId' not in line)
trainSet = trainSet.map(lambda line: (int(line[0]), int(line[1]), int(line[2])))

#pipelinedTrainsetfromUB
trainSetUB = sc.textFile("../trainPipelined.csv")
trainSetUB = trainSetUB.map(lambda l: l.split(','))
trainSetUB = trainSetUB.filter(lambda line: 'userId' not in line)
trainSetUB = trainSetUB.map(lambda line: (int(line[0]), int(line[1]), int(line[2])))

trainSet = trainSet.union(trainSetUB)
#FOR THE TOP POP
itemSet = trainSet.map(lambda x: (x[1], x[2]))
itemsCount = trainSet.map(lambda x: (x[1],1)).reduceByKey(lambda x,y : x + y)
itemsCount_dict = itemsCount.collectAsMap()
#-----------------------------------------
featureFreq = icm.map(lambda x: (x[1],1)).reduceByKey(lambda x, y: x + y)
featureFreqDict = featureFreq.collectAsMap()
prodCount= icm.map(lambda x: x[0]).distinct().count()
featureIdf = featureFreq.map(lambda x: (x[0],np.log10(prodCount/x[1])))
featureIdfDict = featureIdf.collectAsMap()

targetUsers = sc.textFile("../target_user.csv").filter(lambda x: "userId" not in x).map(lambda x: int(x))
targets=targetUsers.collect()

######
#TEST COMPUTING THE PREDICTION NORMALIZING BY THE N OF FEATURES AND NOT THE SQRT OF IT
norms = icm.map(lambda x: (x[0],1))\
                .reduceByKey(lambda x, y: x+y).mapValues(lambda x: np.sqrt(x))\
                .collectAsMap()

normalized = icm.map(lambda x: (x[0], x[1], x[2]/norms[x[0]]))

In [4]:
print(trainSetUB.count())

230048


In [5]:
#Just consider the row of the users to predict
#IF you0re asking, only the idf depends on all the training set, in fact it is computed before reducing the trainset
trainSet=trainSet.filter(lambda x: x[0] in targets)
print(trainSet.count())
data = trainSet.map(lambda x: x[2]).collect()
rows = trainSet.map(lambda x: x[0]).collect()
cols = trainSet.map(lambda x: x[1]).collect()
data.append(0)
rows.append(15364)
cols.append(37142)
userItem=csr_matrix((data,(rows,cols)))
print("userItem shape:",userItem.shape)
data = normalized.map(lambda x: x[2]).collect()
rows = normalized.map(lambda x: x[0]).collect()
cols = normalized.map(lambda x: x[1]).collect()
data.append(0)
rows.append(37142)
cols.append(80)
itemFeature = csc_matrix((data,(rows,cols)))
print("itemFeat shape:",itemFeature.shape)

105691
userItem shape: (15365, 37143)
itemFeat shape: (37143, 19716)


In [6]:
userFeature = userItem.dot(itemFeature)
userFeature.shape

(15365, 19716)

In [7]:
data = []
rows = []
cols = []
for f in featureIdfDict.keys():
    data.append(featureIdfDict[f])
    cols.append(f)
    rows.append(f)
featureIdf = csr_matrix((data,(rows,cols)))
featureIdf.shape

(19716, 19716)

In [8]:
userProfile = userFeature.dot(featureIdf)
prediction = userProfile.dot(itemFeature.transpose())
prediction.shape

(15365, 37143)

In [9]:
numberOfRecommendations=5
#TOP POPULAR
cost=8
avgRatings=itemSet.reduceByKey(lambda x,y: x+y)
avgRatings=avgRatings.map(lambda x: (x[0],x[1]/(itemsCount_dict[x[0]]+cost)))
avgRatings.take(5)
itemOrderByPop=avgRatings.sortBy(lambda x: x[1], ascending=False)
itemPop = np.array(itemOrderByPop.map(lambda x: x[0]).collect())
seenItems= trainSet.map(lambda x: (x[0],[x[1]])).reduceByKey(lambda x,y: x + y)
seenItemsDict=seenItems.collectAsMap()
#--------------------------------------------------------------

def recommendTopPop(user_id, removeSeen=True):
    seenItems = np.array(seenItemsDict[user_id])
    recommendedList = itemPop
    if(removeSeen):
        unseen_mask = np.in1d(recommendedList, seenItems, invert=True)
        recommendedList = recommendedList[unseen_mask]       
    return recommendedList[:numberOfRecommendations]


def fillWithTopPop(recommended,user):
    TopPop=recommendTopPop(user)
    for i in range (numberOfRecommendations-len(recommended)):
        recommended.append(TopPop[i])
    return recommended


def getRecommended(user):
    recommended = []
    itemsPred = prediction.getrow(user).toarray()[0]

    for i in range(0,len(itemsPred)):
        if(itemsPred[i]!=0):
                if i not in seenItemsDict[user]:
                    recommended.append((i, itemsPred[i]))
    recommended.sort(key = lambda x: -x[1])
    recommended=recommended[:numberOfRecommendations]
    print(recommended)
    recommendedItems = list(map(lambda x: x[0], recommended))
    if(len(recommendedItems)<numberOfRecommendations):
        recommendedItems=fillWithTopPop(recommendedItems, user)
    return recommendedItems    



In [10]:
print("After function:")
print(getRecommended(100))
getRecommended(151)

After function:
[(36377, 25.698367404324504), (15930, 25.45347611076965), (21561, 24.471332289902705), (1304, 22.846679971172573), (14351, 22.846679971172573)]
[36377, 15930, 21561, 1304, 14351]
[(35775, 31.264552542429254), (6155, 30.310606017029727), (15221, 29.258161523633575), (12997, 28.900662738539594), (15611, 28.50277867087814)]


[35775, 6155, 15221, 12997, 15611]

In [11]:
f=open("predictionsPip.csv",'w')
i=0
for user in targets:
    print(user)
    f.write("{}".format(user)+',')
    rec=getRecommended(user)
    print(rec)
    for prod in rec:
        f.write(str(prod) +' ')
    f.write('\n')
    i=i+1
    print(i,"of", len(targets), "written")
f.close()

4
[(32578, 10.645831795733727), (35061, 10.146160793559531), (98, 8.3772621380919716), (30408, 8.0441798186082032), (10129, 7.8069883454017415)]
[32578, 35061, 98, 30408, 10129]
1 of 4196 written
5
[(12028, 30.63418715441334), (21884, 30.63418715441334), (34685, 30.63418715441334), (19330, 29.180576843709471), (19372, 29.180576843709471)]
[12028, 21884, 34685, 19330, 19372]
2 of 4196 written
8
[(1304, 36.153111982357316), (14351, 36.153111982357316), (16271, 35.478935659589652), (16826, 35.478935659589652), (17067, 35.31524816091796)]
[1304, 14351, 16271, 16826, 17067]
3 of 4196 written
9
[(30287, 33.874693142087061), (16687, 30.392811408345693), (10059, 29.41605575631748), (10202, 28.872916019199899), (16965, 28.872916019199899)]
[30287, 16687, 10059, 10202, 16965]
4 of 4196 written
13
[(13098, 28.571920913520184), (15266, 26.256174653544278), (1247, 25.049490427696547), (33838, 23.372544530614956), (4455, 23.305795954491259)]
[13098, 15266, 1247, 33838, 4455]
5 of 4196 written
18
[(3

43 of 4196 written
144
[(14079, 7.4648945931547415), (36496, 7.2786841081793821), (1756, 7.2367887119995338), (31272, 7.2367887119995338), (34813, 7.0598560447574288)]
[14079, 36496, 1756, 31272, 34813]
44 of 4196 written
145
[(18439, 5.8513579763510375), (31345, 5.8513579763510375), (20496, 5.6710027118602433), (13040, 5.4173048254702305), (25777, 5.4173048254702305)]
[18439, 31345, 20496, 13040, 25777]
45 of 4196 written
146
[(21412, 23.69013692519853), (8394, 19.257951046927268), (12873, 12.020735593271191), (3398, 10.855795130376684), (17648, 9.1120084289394558)]
[21412, 8394, 12873, 3398, 17648]
46 of 4196 written
151
[(35775, 31.264552542429254), (6155, 30.310606017029727), (15221, 29.258161523633575), (12997, 28.900662738539594), (15611, 28.50277867087814)]
[35775, 6155, 15221, 12997, 15611]
47 of 4196 written
152
[(72, 376.41873549610307), (5280, 376.41873549610307), (5457, 376.41873549610307), (9489, 376.41873549610307), (10465, 376.41873549610307)]
[72, 5280, 5457, 9489, 1046

288
[(21874, 8.4499928721136151), (36587, 8.4030205210204958), (4034, 8.2619487480718092), (18106, 7.5740168089260846), (20101, 7.2002211217607357)]
[21874, 36587, 4034, 18106, 20101]
87 of 4196 written
294
[(24105, 33.921508264478717), (30087, 33.752441838204312), (12028, 32.643907720675159), (21884, 32.643907720675159), (34685, 32.643907720675159)]
[24105, 30087, 12028, 21884, 34685]
88 of 4196 written
296
[(18741, 17.466330073765672), (35061, 8.458695114002925), (17644, 7.3563798741746478), (25694, 6.4642543540571031), (8809, 6.2567686537326468)]
[18741, 35061, 17644, 25694, 8809]
89 of 4196 written
299
[(36855, 36.286589786774762), (12028, 36.141248591522341), (21884, 36.141248591522341), (34685, 36.141248591522341), (33143, 35.787654165096576)]
[36855, 12028, 21884, 34685, 33143]
90 of 4196 written
301
[(22457, 26.216525863837919), (1963, 24.996672967653204), (19161, 24.789420713559299), (2157, 24.458845408297957), (4357, 24.458845408297957)]
[22457, 1963, 19161, 2157, 4357]
91 of

[(11758, 45.057131479044024), (19330, 39.071717788161237), (19372, 39.071717788161237), (25142, 39.071717788161237), (12028, 38.446007062940346)]
[11758, 19330, 19372, 25142, 12028]
131 of 4196 written
441
[(2157, 38.660024116233984), (4357, 38.660024116233984), (7586, 38.660024116233984), (13017, 38.660024116233984), (16278, 38.660024116233984)]
[2157, 4357, 7586, 13017, 16278]
132 of 4196 written
443
[(15973, 37.03711291862404), (35294, 34.347562117758308), (6117, 34.1209131017837), (26360, 34.1209131017837), (8216, 33.675420488491099)]
[15973, 35294, 6117, 26360, 8216]
133 of 4196 written
445
[(15971, 28.808036497851901), (15973, 28.439894775049417), (20941, 28.439894775049417), (33266, 27.915737233305819), (12028, 27.403278128610218)]
[15971, 15973, 20941, 33266, 12028]
134 of 4196 written
449
[(7592, 25.345909885561809), (22002, 24.762242848991569), (5128, 24.221092236504326), (5647, 24.195420379242204), (35461, 22.98071673872068)]
[7592, 22002, 5128, 5647, 35461]
135 of 4196 writ

[23336, 16803, 16687, 34602, 14734]
171 of 4196 written
592
[(15973, 39.167289692836427), (20941, 39.167289692836427), (33266, 38.57761245837488), (17011, 36.155163230161939), (13220, 34.993141091485541)]
[15973, 20941, 33266, 17011, 13220]
172 of 4196 written
595
[(35695, 27.200667738132637), (12028, 25.080795697154649), (21884, 25.080795697154649), (34685, 25.080795697154649), (6663, 25.077563857423598)]
[35695, 12028, 21884, 34685, 6663]
173 of 4196 written
599
[(4837, 31.603806221092196), (26761, 30.07565567050483), (27952, 30.07565567050483), (11417, 29.976513175203145), (25458, 29.976513175203145)]
[4837, 26761, 27952, 11417, 25458]
174 of 4196 written
600
[(16632, 18.596140545584742), (1489, 14.959859960731777), (27071, 13.207915095901301), (3067, 11.430566301599139), (22061, 11.374770517333168)]
[16632, 1489, 27071, 3067, 22061]
175 of 4196 written
603
[(8336, 31.477726767478316), (5866, 30.504762522119513), (21796, 30.504762522119513), (25050, 30.22810050533629), (2157, 30.084

...

[(25049, 28.426353678426928), (35294, 28.384311232831749), (3380, 26.692772715277574), (29413, 25.361029730277799), (14532, 25.16744622480109)]
[25049, 35294, 3380, 29413, 14532]
4022 of 4196 written
14760
[(15090, 6.2108621535917434), (37033, 5.3777729383943278), (18212, 5.2968304425959731), (22927, 5.2968304425959731), (26366, 4.9705872540987954)]
[15090, 37033, 18212, 22927, 26366]
4023 of 4196 written
14766
[(11707, 39.207207753026225), (28750, 34.677204985642334), (15973, 33.179404627276384), (20941, 33.179404627276384), (33266, 32.655247085532778)]
[11707, 28750, 15973, 20941, 33266]
4024 of 4196 written
14769
[(26473, 37.719137449957891), (8336, 31.151508967006549), (7660, 30.207804286686706), (15627, 30.207804286686706), (25852, 30.207804286686706)]
[26473, 8336, 7660, 15627, 25852]
4025 of 4196 written
14773
[(6663, 38.209734885176132), (11703, 38.209734885176132), (12028, 38.177029917862868), (21884, 38.177029917862868), (34685, 38.177029917862868)]
[6663, 11703, 12028, 21884

[(32456, 29.876658063285753), (33215, 29.685213511884672), (4009, 29.612146125868367), (21399, 29.612146125868367), (30997, 29.612146125868367)]
[32456, 33215, 4009, 21399, 30997]
4062 of 4196 written
14890
[(35967, 27.737557345015581), (36982, 27.257888160431524), (33595, 26.950445704649695), (23464, 26.556106854174438), (7906, 26.465528698437904)]
[35967, 36982, 33595, 23464, 7906]
4063 of 4196 written
14891
[(35953, 32.038807610996706), (24309, 28.232584739535113), (30300, 28.232584739535113), (30942, 25.535117460508523), (32520, 24.744519172844001)]
[35953, 24309, 30300, 30942, 32520]
4064 of 4196 written
14893
[(23453, 32.402573667275881), (27258, 31.132695248825652), (29847, 31.132695248825652), (30503, 31.132695248825652), (35440, 31.132695248825652)]
[23453, 27258, 29847, 30503, 35440]
4065 of 4196 written
14899
[(11707, 36.867439853841674), (28750, 32.33743708645779), (36855, 31.452759250627572), (33215, 29.203059753827262), (37096, 28.802534128781424)]
[11707, 28750, 36855, 3

4104 of 4196 written
15026
[(35294, 39.109703880349755), (15973, 38.1730940920393), (20941, 38.1730940920393), (33266, 37.583416857577745), (6117, 36.392716923154957)]
[35294, 15973, 20941, 33266, 6117]
4105 of 4196 written
15027
[(8877, 31.654674463902097), (26473, 31.233678763702638), (2398, 30.545086923849929), (26741, 30.303242863353937), (7048, 27.618744722311163)]
[8877, 26473, 2398, 26741, 7048]
4106 of 4196 written
15028
[(15973, 32.197518398888057), (20941, 32.197518398888057), (33266, 31.673360857144459), (15971, 30.354367995903878), (12028, 29.582666241614767)]
[15973, 20941, 33266, 15971, 12028]
4107 of 4196 written
15030
[(4064, 34.013003442626868), (12997, 33.997877386026595), (5866, 33.52374980905094), (21796, 33.52374980905094), (15973, 33.023638013479214)]
[4064, 12997, 5866, 21796, 15973]
4108 of 4196 written
15031
[(28884, 0.96555954544578892), (36212, 0.93065182138054792), (30559, 0.92466404487856746), (21286, 0.89393443470095968), (8037, 0.84409892582326784)]
[2888

[(65, 28.196521583866257), (15761, 28.196521583866257), (20525, 28.196521583866257), (23241, 28.196521583866257), (2455, 27.219400002657643)]
[65, 15761, 20525, 23241, 2455]
4146 of 4196 written
15168
[(7274, 16.550730929439371), (8759, 16.550730929439371), (9382, 8.6864048724004501), (32005, 8.6864048724004501), (2217, 8.6273049999335392)]
[7274, 8759, 9382, 32005, 2217]
4147 of 4196 written
15176
[(24932, 29.465471324877182), (6663, 26.930813119695394), (11703, 26.930813119695394), (4850, 26.907176281098693), (5086, 26.907176281098693)]
[24932, 6663, 11703, 4850, 5086]
4148 of 4196 written
15180
[(2157, 29.900431419081727), (4357, 29.900431419081727), (7586, 29.900431419081727), (13017, 29.900431419081727), (16278, 29.900431419081727)]
[2157, 4357, 7586, 13017, 16278]
4149 of 4196 written
15183
[(816, 24.356883622483636), (3048, 20.585323825469366), (34873, 15.319399454818795), (36161, 11.885203407701042), (12552, 9.7796990687146987)]
[816, 3048, 34873, 36161, 12552]
4150 of 4196 wri

4186 of 4196 written
15330
[(26681, 17.141513064164343), (28083, 14.932106678777311), (45, 11.749476833801801), (34554, 10.997000705761906), (25412, 10.784045179627908)]
[26681, 28083, 45, 34554, 25412]
4187 of 4196 written
15331
[(20110, 42.38842816545165), (15971, 36.094050539374862), (12997, 34.621423098749943), (36855, 32.576340065436547), (12028, 31.904903975173557)]
[20110, 15971, 12997, 36855, 12028]
4188 of 4196 written
15332
[(569, 22.733692935351634), (28898, 20.114492763152075), (11871, 19.675763894484078), (12997, 19.53447558999823), (19314, 19.303320307033353)]
[569, 28898, 11871, 12997, 19314]
4189 of 4196 written
15334
[(32935, 23.654449342942659), (33595, 21.276499830795945), (13307, 19.789261624091758), (25253, 19.729374747643526), (35205, 19.428388334454496)]
[32935, 33595, 13307, 25253, 35205]
4190 of 4196 written
15337
[(14345, 33.587949139039559), (2882, 33.583694601097633), (6365, 33.583694601097633), (7250, 33.583694601097633), (13403, 33.583694601097633)]
[14345

In [18]:
print("Result of getRecommended:\n",getRecommended(151))
print("Top 5 from the matrix: " ,prediction.getrow(151).toarray()[0])

[(8013, 27.457348334313153), (30796, 23.836096268859752), (12535, 18.652665545838573), (4223, 13.12635525344616), (11554, 13.12635525344616)]
Result of getRecommended:
 [8013, 30796, 12535, 4223, 11554]
Top 5 from the matrix:  [ 0.          3.52206684  6.39351585 ...,  0.58138567  4.65589395  0.        ]


In [19]:
print("Result of getRecommended:\n",getRecommended(100))
print("Top 5 from the matrix: " ,prediction.getrow(100).toarray()[0])

[(12535, 18.652665545838573), (4223, 13.12635525344616), (11554, 13.12635525344616), (20532, 13.00817618709717), (19267, 12.695308889326265)]
Result of getRecommended:
 [12535, 4223, 11554, 20532, 19267]
Top 5 from the matrix:  [ 0.          2.36576972  3.58037592 ...,  0.          1.72884326  0.        ]
